## Welcome to your notebook.


#### Run this cell to connect to your GIS and get started:

Note: I've had issues where some components were installed on the normal channel first. Solution is either wait for the server to timeout or (maybe) do a full conda install without sys

In [1]:
import sys
!{sys.executable} -m conda install -y osmnx -c conda-forge
#!{sys.executable} -m conda install -y pyogrio -c conda-forge

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.10.3
  latest version: 23.3.0

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



#### Orgin and Destination Table from points

In [2]:
def sample(df, n=100):
    return df.sample(n=n, axis='index')

In [3]:
import numpy as np
import osmnx as ox
import geopandas
import pandas
import datetime

In [4]:
geopandas.__version__

'0.12.2'

In [5]:
import fiona
fiona.__version__

ERROR 1: PROJ: proj_create_from_database: SQLite error on SELECT name, type, coordinate_system_auth_name, coordinate_system_code, datum_auth_name, datum_code, area_of_use_auth_name, area_of_use_code, text_definition, deprecated FROM geodetic_crs WHERE auth_name = ? AND code = ?: no such column: area_of_use_auth_name
PROJ: proj_create_from_database: SQLite error on SELECT name, type, coordinate_system_auth_name, coordinate_system_code, datum_auth_name, datum_code, area_of_use_auth_name, area_of_use_code, text_definition, deprecated FROM geodetic_crs WHERE auth_name = ? AND code = ?: no such column: area_of_use_auth_name


'1.8.20'

In [6]:
import pyproj
pyproj.__version__

'2.6.1.post1'

In [7]:
# User variables
mode = 'walk'  # 'walk' or 'drive'
origins_shp = '/arcgis/home/All_Parcels_Tracts_within_Centroids2_1222022.shp'
destinations_shp = '/arcgis/home/ParksandRec_Tracts_within_12232022.shp'

In [8]:
# read shp to geodataframe
origins_all = geopandas.read_file(origins_shp)
destinations_all = geopandas.read_file(destinations_shp)

CRSError: Invalid projection: PROJCS["USA_Contiguous_Albers_Equal_Area_Conic_USGS_version",GEOGCS["NAD83",DATUM["North_American_Datum_1983",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6269"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",23],PARAMETER["longitude_of_center",-96],PARAMETER["standard_parallel_1",29.5],PARAMETER["standard_parallel_2",45.5],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]: (Internal Proj Error: proj_create: SQLite error on SELECT name, ellipsoid_auth_name, ellipsoid_code, prime_meridian_auth_name, prime_meridian_code, area_of_use_auth_name, area_of_use_code, deprecated FROM geodetic_datum WHERE auth_name = ? AND code = ?: no such column: area_of_use_auth_name)

In [ ]:
# Get subset
origins_sub = sample(origins_all, n=10)
destinations_sub = sample(destinations_all, n=10)

In [ ]:
def OD_bbox(origins, destinations):
    # Get bbox
    # minx, miny, maxx, maxy
    #expected maxy, miny, maxx, minx
    # TODO: add 1000m to bbox (inline with graph from point)
    #clean_periphery uses 500m
    o_bbox = origins.to_crs(4326).total_bounds
    d_bbox = destinations.to_crs(4326).total_bounds
    #od_bbox = [min(o_bbox[0], d_bbox[0]), min(o_bbox[1], d_bbox[1]), max(o_bbox[2], d_bbox[2]), max(o_bbox[3], d_bbox[3])]
    return {'minx': min(o_bbox[0], d_bbox[0]), 
            'miny': min(o_bbox[1], d_bbox[1]),
            'maxx': max(o_bbox[2], d_bbox[2]), 
            'maxy': max(o_bbox[3], d_bbox[3])
            }

In [ ]:
def default_network(origins, destinations):
    bbox_dict = OD_bbox(origins, destinations)
    # Get ox network
    G = ox.graph_from_bbox(bbox_dict['maxy'],
                           bbox_dict['miny'],
                           bbox_dict['maxx'],
                           bbox_dict['minx'],
                           network_type="drive")
    # impute speed on all edges missing data
    G = ox.add_edge_speeds(G)
    # calculate travel time (seconds) for all edges
    G = ox.add_edge_travel_times(G)
    return G

In [ ]:
def on_network(points, G):
    Gp = ox.project_graph(G)  # Faster to run this once for O-D
    # Get CRS (should be location based UTM)
    crs = ox.graph_to_gdfs(Gp, nodes=False).crs
    points = points.to_crs(crs)
    X_series = points['geometry'].x
    Y_series = points['geometry'].y
    
    nodes, dists = ox.nearest_nodes(Gp, X_series, Y_series, return_dist=True)
    return pandas.DataFrame({'Nearest_Node': nodes,
                             'Offset_Distance': dists})

In [ ]:
def OD_matrix(origin_nodes, destination_nodes, graph, weight="length"):
    assert weight in ["length", "travel_time"], 'Weight not recognized'
    routes = []
    routes_d = []
    for i, orig in enumerate(origin_nodes):
        for j, dest in enumerate(destination_nodes):
            route = ox.shortest_path(G, orig, dest, weight=weight)
            routes.append(route)
            routes_d.append(
                {
                    'Origin': i,
                    'Destination': j,
                    'Distance (m)': int(sum(ox.utils_graph.get_route_edge_attributes(G, route, "length"))),
                    'Travel_Time (s)': int(sum(ox.utils_graph.get_route_edge_attributes(G, route, "travel_time"))),
                }
            )
            #ox.utils_graph.get_route_edge_attributes(G, route) # all attribtues
            #route_length_col.append()
            #route_time_col.append(sum(ox.utils_graph.get_route_edge_attributes(G, route, "travel_time")))
    df = pandas.DataFrame(routes_d)
    df['Travel_Time'] = [str(datetime.timedelta(seconds=sec)) for sec in df['Travel_Time (s)']]
    return df, routes

In [ ]:
# Get network
G = default_network(origins_sub, destinations_sub)

In [ ]:
# Locate origins on network
o_df = on_network(origins_sub, G)
o_df

In [ ]:
# Locate destinations on network
d_df = on_network(destinations_sub, G)
d_df

In [ ]:
# find the shortest path (by distance) between first nodes then plot it
orig = o_df['Nearest_Node'][0]
dest = d_df['Nearest_Node'][0]
route = ox.shortest_path(G, orig, dest, weight="length")
fig, ax = ox.plot_graph_route(G, route, route_color="y", route_linewidth=6, node_size=0)

In [ ]:
df, routes = OD_matrix(o_df['Nearest_Node'], d_df['Nearest_Node'], G)

In [ ]:
df

In [ ]:
# Plot first route
fig, ax = ox.plot_graph_route(G, routes[0], route_color="y", route_linewidth=6, node_size=0)

#### Tutorial

In [ ]:
import numpy as np
import osmnx as ox

%matplotlib inline
np.random.seed(0)
ox.__version__

In [ ]:
place = "Pensacola, Florida, USA"
G = ox.graph_from_place(place, network_type="drive")
Gp = ox.project_graph(G)

In [ ]:
# randomly sample n points spatially-constrained to the network's geometry
points = ox.utils_geo.sample_points(ox.get_undirected(Gp), n=100)
X = points.x.values
Y = points.y.values
X0 = X.mean()
Y0 = Y.mean()

In [ ]:
# NOTES: appears to be projected to UTM?
X[0]

In [ ]:
Y[0]

In [ ]:
# find each nearest node to several points, and optionally return distance
nodes, dists = ox.nearest_nodes(Gp, X, Y, return_dist=True)

In [ ]:
# or, find the nearest node to a single point
node = ox.nearest_nodes(Gp, X0, Y0)
node

In [ ]:
# in feet? Makes sense it would be real close since the points are along the network
dists[0]

In [ ]:
# find each nearest edge to several points, and optionally return distance
edges, dists = ox.nearest_edges(Gp, X, Y, return_dist=True)

In [ ]:
# find the nearest edge to a single point
edge = ox.nearest_edges(Gp, X0, Y0)
edge

In [ ]:
# find the shortest path (by distance) between these nodes then plot it
orig = list(G)[0]
dest = list(G)[120]
route = ox.shortest_path(G, orig, dest, weight="length")
fig, ax = ox.plot_graph_route(G, route, route_color="y", route_linewidth=6, node_size=0)

In [ ]:
route

In [ ]:
routes = ox.k_shortest_paths(G, orig, dest, k=30, weight="length")
fig, ax = ox.plot_graph_routes(G, list(routes), route_colors="y", route_linewidth=4, node_size=0)

In [ ]:
# impute speed on all edges missing data
G = ox.add_edge_speeds(G)

# calculate travel time (seconds) for all edges
G = ox.add_edge_travel_times(G)

In [ ]:
# see mean speed/time values by road type
edges = ox.graph_to_gdfs(G, nodes=False)
edges["highway"] = edges["highway"].astype(str)
edges.groupby("highway")[["length", "speed_kph", "travel_time"]].mean().round(1)

In [ ]:
# same thing again, but this time pass in a few default speed values (km/hour)
# to fill in edges with missing `maxspeed` from OSM
hwy_speeds = {"residential": 35, "secondary": 50, "tertiary": 60}
G = ox.add_edge_speeds(G, hwy_speeds)
G = ox.add_edge_travel_times(G)

In [ ]:
# Unlcear why relationship between speed_kph, maxspeed and which is edge_speeds
edges[['speed_kph', 'maxspeed']]

In [ ]:
# calculate two routes by minimizing travel distance vs travel time
#orig = list(G)[1]
#dest = list(G)[120]
route1 = ox.shortest_path(G, orig, dest, weight="length")
#route2 = ox.shortest_path(G, orig, dest, weight="travel_time")

In [ ]:
route1

In [ ]:
route2 = ox.shortest_path(G, orig, dest, weight="travel_time")

In [ ]:
route2

In [ ]:
# plot the routes
fig, ax = ox.plot_graph_routes(
    G, routes=[route1, route2], route_colors=["r", "y"], route_linewidth=6, node_size=0
)

In [ ]:
# compare the two routes
route1_length = int(sum(ox.utils_graph.get_route_edge_attributes(G, route1, "length")))
route2_length = int(sum(ox.utils_graph.get_route_edge_attributes(G, route2, "length")))
route1_time = int(sum(ox.utils_graph.get_route_edge_attributes(G, route1, "travel_time")))
route2_time = int(sum(ox.utils_graph.get_route_edge_attributes(G, route2, "travel_time")))
print("Route 1 is", route1_length, "meters and takes", route1_time, "seconds.")
print("Route 2 is", route2_length, "meters and takes", route2_time, "seconds.")

In [ ]:
# point geo to send to other services (not in edges or nodes?)
orig